In [2]:
from pandas import read_csv 
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = read_csv('international-airline-passengers.csv', usecols=[1])
df.describe()

In [3]:
values = df.values.astype('float32')
values.shape

NameError: name 'df' is not defined

In [ ]:
train_size = int((values.shape[0] * 0.67))
test_size = values.shape[0]  - train_size
train = values[0:train_size]
test = values[train_size:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range =(0,1))
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [ ]:
def create_dataset(data, k):
    dataX, dataY = [],[]
    for i in range(data.shape[0] - k):
        x = data[i:i + k, 0]
        y = data[i + k, 0]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 12
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
trainX.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
model = Sequential()
model.add(SimpleRNN(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size = 1)

In [ ]:
#predicting on the test data 
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict).ravel() #unscaling it 

#predicting on the train data 
trainPredict = model.predict(trainX)
trainPredict = scaler.inverse_transform(trainPredict).ravel() # Ravel converts to 1-D

testTrue = scaler.inverse_transform([testY]).ravel() # Instead of test predict we are using testY ...and testY and trainY are one dimensional 
trainTrue = scaler.inverse_transform([trainY]).ravel() # By putting the square bracket we are making it two dimensional as they are suppose to be two dimensional

testPredict.shape, testTrue.shape

In [ ]:
# now we are plotting it 
from matplotlib import pyplot as plt

In [ ]:
# plotting the training data 
plt.plot(trainTrue, c = 'g')
#gren one is the true data

# plotting the training predicit data 
plt.plot(trainPredict, c = 'b')
#blue one is the predicted data 

plt.show()

In [ ]:
combinedPredicted = np.concatenate((trainPredict, testPredict)) # we are combining training and testing both together
# we concatenated the train predict and the test predict
combinedTrue = np.concatenate((trainTrue, testTrue))
# here we have combined the true test and train values 

#now plotting them 
plt.plot(combinedTrue, c = 'g')
plt.plot(combinedPredicted, c = 'b')
plt.show()